In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from phik import phik_matrix

In [2]:
plt.style.use('dark_background')

In [3]:
df = pd.read_csv('total_df.csv')

In [4]:
df.head()

,vacancy_id,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_salary_from,vacancy_salary_to,vacancy_salary_currency,vacancy_salary_gross,...,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status,target
0,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Казанский Авиационный Институт'],NaN,NaN,65.0,19 лет,228.0,Москва,Мужчина,Рассматривает предложения,1
1,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,"['ООО ""Открытый Учебный Центр СофтБаланс"", г. ...","['ООО ""Открытый Учебный Центр СофтБаланс"", г. ...",NaN,43.0,17 лет 4 месяца,208.0,Москва,Мужчина,Рассматривает предложения,1
2,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Орский государственный педагогический инстит...,NaN,200 000 ₽ на руки,52.0,30 лет,360.0,Москва,Женщина,NaN,1
3,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Красноярский государственный университет'],NaN,500 000 ₽ на руки,56.0,29 лет 8 месяцев,356.0,Красноярск,Мужчина,Рассматривает предложения,1
4,126167948,Разработчик SAP ABAP,Москва,Более 6 лет,Полная занятость,Удаленная работа,300000.0,NaN,RUR,False,...,['Белоруский Гос. Университет Информатики и Ра...,"['SAP CIS, SAP XI', 'Школа Логистики МАДИ', 'S...",NaN,48.0,25 лет 1 месяц,301.0,Moscow,Male,NaN,1


In [5]:
df.shape

(59610, 28)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59610 entries, 0 to 59609
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   vacancy_id                             59610 non-null  int64  
 1   vacancy_name                           59610 non-null  object 
 2   vacancy_area                           59610 non-null  object 
 3   vacancy_experience                     59610 non-null  object 
 4   vacancy_employment                     59610 non-null  object 
 5   vacancy_schedule                       59610 non-null  object 
 6   vacancy_salary_from                    14031 non-null  float64
 7   vacancy_salary_to                      11257 non-null  float64
 8   vacancy_salary_currency                17597 non-null  object 
 9   vacancy_salary_gross                   17597 non-null  object 
 10  vacancy_description                    59610 non-null  object 
 11  re

In [7]:
df.duplicated().sum()

np.int64(0)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Дубликатов не выявлено.
</div>

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Посмотрим на пропуски отдельно по категориальным и числовым признакам.
</div>

In [8]:
nan_df = pd.DataFrame({'Количество пропусков': df.isna().sum(), 'Доля пропусков': (df.isna().sum() / len(df)).round(2)})

In [9]:
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(include=['object']).columns

In [10]:
nan_df.loc[list(cat_cols)].sort_values('Доля пропусков', ascending=False)

,Количество пропусков,Доля пропусков
vacancy_salary_currency,42013,0.70
vacancy_salary_gross,42013,0.70
resume_applicant_status,40095,0.67
resume_courses,35899,0.60
resume_salary,26695,0.45
resume_skills,15136,0.25
resume_education,1520,0.03
resume_last_experience_description,1209,0.02
resume_total_experience,1009,0.02
resume_last_company_experience_period,1196,0.02


In [11]:
nan_df.loc[list(num_cols)].sort_values('Доля пропусков', ascending=False)

,Количество пропусков,Доля пропусков
vacancy_salary_to,48353,0.81
vacancy_salary_from,45579,0.76
resume_age,3281,0.06
resume_experience_months,1009,0.02
vacancy_id,0,0.00
resume_id,0,0.00
target,0,0.00


<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Посмотрим на пропуски отдельно по категориальным и числовым признакам.
</div>

In [12]:
df[cat_cols] = df[cat_cols].fillna('NDT')

In [27]:
df.loc[df['resume_experience_months'].isna(), 'resume_last_experience_description'].unique()

array(['NDT'], dtype=object)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Т.к. у признаков vacancy_salary_to и vacancy_salary_from примерно 80% пропусков, то можно их удалить. Признак resume_age заполним средним значением, а resume_experience_months заполним нулем, т.к. видно, что последнего места работы нет.
</div>

In [26]:
df['resume_age'] = df['resume_age'].fillna('mean')
df['resume_experience_months'] = df['resume_experience_months'].fillna(0)

In [13]:
df = df.drop(['vacancy_id', 'resume_id', 'vacancy_salary_to', 'vacancy_salary_from'], axis=1)

In [14]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
vacancy_name,59610,2720,DevOps-инженер,1539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_area,59610,23,Москва,58841,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_experience,59610,4,От 3 до 6 лет,32229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_employment,59610,3,Полная занятость,57821,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_schedule,59610,5,Удаленная работа,29976,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_salary_from,14031.0,NaN,NaN,NaN,171857.380942,114919.572917,25.0,100000.0,150000.0,220000.0,1000000.0
vacancy_salary_to,11257.0,NaN,NaN,NaN,219639.788576,125977.930942,35.0,150000.0,200000.0,300000.0,1000000.0
vacancy_salary_currency,59610,4,NDT,42013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_salary_gross,59610,3,NDT,42013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vacancy_description,59610,3382,ООО «Исратэк» — 30 лет в производстве самоклея...,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Добавим дополнительный столбец с опытом работы в последней компании в месяцах для удобства
</div>

In [15]:
def experience_to_months(experience_text):
    months = 0
    # Опыт в годах
    years_match = re.search(r'(\d+)\s*год', experience_text)
    if years_match:
        months += int(years_match.group(1)) * 12
    
    years_match = re.search(r'(\d+)\s*лет', experience_text)
    if years_match:
        months += int(years_match.group(1)) * 12
    
    # Опыт в месяцах
    months_match = re.search(r'(\d+)\s*месяц', experience_text)
    if months_match:
        months += int(months_match.group(1))
    
    return months if months > 0 else np.nan

In [16]:
df['resume_last_company_experience_months'] = df['resume_last_company_experience_period'].apply(experience_to_months)

In [22]:
df.loc[df['resume_last_company_experience_period'] == 'NDT', 'resume_last_experience_description'].unique()

array(['NDT'], dtype=object)

<div style="background-color: #98FB98; color: black; padding: 10px; border-radius: 5px;">
Т.к. в названии компании стоит NDT, можно столбец resume_last_company_experience_months заполнять нулевыми значениями.
</div>

In [23]:
df['resume_last_company_experience_months'] = df['resume_last_company_experience_months'].fillna(0)